# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [20]:
# imports
import os
import json
import tempfile
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import gradio as gr

In [21]:
# constants and environment
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not set. Add it to .env or your environment.")
client = OpenAI()

In [22]:
# system prompt: technical tutor expertise
SYSTEM_PROMPT = """You are an expert technical tutor. You explain programming, computer science, and technology clearly and concisely. Use examples when helpful. If the user asks for the current date or time, use the get_current_time tool. Format code and key terms clearly.

For mathematical or chemical equations, use standard LaTeX in markdown so they render properly:
- Display equations: wrap in double dollar signs on their own line, e.g. $$x^2 + y^2 = z^2$$
- Inline math: use single dollar signs, e.g. $E = mc^2$
- Do not wrap equations in square brackets like [ \\text{...} ]; use $$ ... $$ for display equations instead."""

In [23]:
# tool: get_current_time (demonstrates tool use with GPT)
get_current_time_tool = {
    "type": "function",
    "function": {
        "name": "get_current_time",
        "description": "Get the current date and time. Use when the user asks for the time, date, or 'what time is it'.",
        "parameters": {
            "type": "object",
            "properties": {},
            "additionalProperties": False,
        },
    },
}
TOOLS = [get_current_time_tool]


def handle_tool_calls(message):
    """Execute tool calls and return tool response messages."""
    responses = []
    for tc in message.tool_calls:
        if tc.function.name == "get_current_time":
            print("Tool call received....:", tc.function.name, flush=True)
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S %Z")
            responses.append({"role": "tool", "content": now, "tool_call_id": tc.id})
    return responses

In [24]:
def chat_streaming(history, model_name):
    """Stream chat response. Supports GPT (with tools) and Llama. Yields (history_with_partial_assistant_message)."""
    history = list(history) if history else []
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + [
        {"role": h["role"], "content": h["content"]} for h in history
    ]

    if model_name == MODEL_GPT:
        # GPT: use tools, then stream final reply (or stream directly if no tool call)
        response = client.chat.completions.create(
            model=MODEL_GPT, messages=messages, tools=TOOLS, stream=False
        )
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            messages.append({"role": "assistant", "content": msg.content or "", "tool_calls": [{"id": tc.id, "type": "function", "function": {"name": tc.function.name, "arguments": tc.function.arguments}} for tc in msg.tool_calls]})
            for r in handle_tool_calls(msg):
                messages.append(r)
            response = client.chat.completions.create(
                model=MODEL_GPT, messages=messages, tools=TOOLS, stream=False
            )
        final_content = response.choices[0].message.content or ""
        # Simulate streaming by yielding progressively
        accumulated = ""
        for char in final_content:
            accumulated += char
            yield history + [{"role": "assistant", "content": accumulated}]
        return

    # Llama (Ollama): stream directly
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True,
    )
    accumulated = ""
    for chunk in stream:
        if "message" in chunk and chunk["message"].get("content"):
            accumulated += chunk["message"]["content"]
            yield history + [{"role": "assistant", "content": accumulated}]

In [25]:
# audio: speech-to-text (Whisper) and text-to-speech (OpenAI TTS)
def transcribe_audio(audio_path):
    """Convert speech to text using OpenAI Whisper."""
    if audio_path is None:
        return ""
    try:
        with open(audio_path, "rb") as f:
            transcript = client.audio.transcriptions.create(model="whisper-1", file=f)
        return transcript.text.strip()
    except Exception as e:
        return f"(Transcription error: {e})"


def text_to_speech(text):
    """Convert assistant reply to speech; returns path to temp MP3 for Gradio."""
    if not text:
        return None
    try:
        fd, path = tempfile.mkstemp(suffix=".mp3")
        os.close(fd)
        with client.audio.speech.with_streaming_response.create(
            model="tts-1", voice="nova", input=text[:4096]
        ) as response:
            response.stream_to_file(path)
        return path
    except Exception as e:
        print(f"TTS error: {e}")
        return None

In [28]:
# Gradio UI: technical Q&A with streaming, model switch, audio in/out
def add_user_and_chat(message, history, model_name, respond_with_audio):
    """Add user message to history, stream assistant reply, optionally generate TTS."""
    if not message or not message.strip():
        return history, None
    is_first_message = not (history or [])
    history = list(history or []) + [{"role": "user", "content": message.strip()}]
    # Only show placeholder on first message so the loader appears on subsequent ones
    if is_first_message:
        yield history + [{"role": "assistant", "content": "..."}], None
    last_assistant_content = ""
    for updated in chat_streaming(history, model_name):
        last_assistant_content = updated[-1]["content"] if updated else ""
        yield updated, None
    if respond_with_audio and last_assistant_content:
        audio_path = text_to_speech(last_assistant_content)
        yield history + [{"role": "assistant", "content": last_assistant_content}], audio_path
    else:
        yield history + [{"role": "assistant", "content": last_assistant_content}], None


def on_audio_upload(audio_path, history, model_name, respond_with_audio):
    """Transcribe audio to text, then run add_user_and_chat."""
    text = transcribe_audio(audio_path)
    if not text or text.startswith("("):
        return history, None
    for hist, aud in add_user_and_chat(text, history, model_name, respond_with_audio):
        yield hist, aud


with gr.Blocks(title="Technical Q&A Assistant") as demo:
    gr.Markdown("## Technical Q&A Assistant\nAsk a technical question; switch models; use voice in/out.")
    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=[MODEL_GPT, MODEL_LLAMA],
            value=MODEL_GPT,
            label="Model",
        )
        respond_with_audio = gr.Checkbox(value=False, label="Respond with audio (TTS)")
    chatbot = gr.Chatbot(height=400, type="messages", label="Chat")
    with gr.Row():
        msg = gr.Textbox(placeholder="Ask a technical question...", label="Message", scale=4)
        send_btn = gr.Button("Send", scale=1)
    audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath", label="Or speak your question")
    audio_output = gr.Audio(label="Assistant reply (audio)", visible=False)
    clear_btn = gr.Button("Clear")

    def clear():
        return [], None, None

    send_btn.click(
        fn=add_user_and_chat,
        inputs=[msg, chatbot, model_dropdown, respond_with_audio],
        outputs=[chatbot, audio_output],
    ).then(lambda: "", None, msg)

    msg.submit(
        fn=add_user_and_chat,
        inputs=[msg, chatbot, model_dropdown, respond_with_audio],
        outputs=[chatbot, audio_output],
    ).then(lambda: "", None, msg)

    audio_input.stop_recording(
        fn=on_audio_upload,
        inputs=[audio_input, chatbot, model_dropdown, respond_with_audio],
        outputs=[chatbot, audio_output],
    )
    audio_input.change(
        fn=on_audio_upload,
        inputs=[audio_input, chatbot, model_dropdown, respond_with_audio],
        outputs=[chatbot, audio_output],
    )

    respond_with_audio.change(
        fn=lambda x: gr.Audio(visible=x),
        inputs=respond_with_audio,
        outputs=audio_output,
    )
    clear_btn.click(fn=clear, inputs=None, outputs=[chatbot, audio_output, audio_input])

In [29]:
demo.queue()
demo.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
